In [ ]:
!pip install torch
!pip install peft transformers accelerate


In [ ]:
import torch
from transformers import BartForConditionalGeneration
from transformers import TrainingArguments
from transformers import Trainer
from peft import get_peft_model, LoraConfig, TaskType

import os
os.environ["WANDB_DISABLED"] = "true"

# Load tokenized data
data = torch.load("train_encodings.pt")
inputs = data["inputs"]
labels = data["labels"]

# Define dataset class
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels["input_ids"][idx])
        return item

# Prepare dataset
train_dataset = QADataset(inputs, labels)

# Continue with model fine-tuning...
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # Specify the task type
    r=8,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to specific layers (query/key/value projections)
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none"  # Bias setup
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results_lora",
    evaluation_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs_lora",
    fp16=True,  # Use mixed precision for faster training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Fine-tune the model with LoRA
trainer.train()

<ipython-input-4-75f36f02f02e>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("/content/train_encodings.pt")
/usr/local/lib/python3.10/dist-packages/hug

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,7.865500


KeyboardInterrupt: 

In [ ]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)
def get_answer(question):
    # Tokenize the question
    inputs = tokenizer(question, return_tensors="pt", max_length=128, truncation=True)

    # Move input tensors to the same device as the model
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Generate the response using the model
    output_ids = model.base_model.generate(inputs["input_ids"], max_length=150, num_beams=4, length_penalty=2.0)

    # Decode the response
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer


# Example interaction

user_question = "What is the use of Guanfacine?"
print(f"Answer: {get_answer(user_question)}")


Answer: Birth Control
